# Making Predictions: An Example Using the Beer Dataset

We will go through the proccess for predicting Y quantities with SIMCA-Q.

We will use a SIMCA project where two OPLS models have been built using the BEER dataset typically used in SIMCA courses. The picture below shows the initial observations and variables of the original dataset used in this project:

![Original Dataset](Dataset1.png)

The SIMCA project file has two OPLS models, both predicting the alcohol content:

![Models](Images/Models.png)